In [1]:
# import os
# # import Google Drive 套件
# from google.colab import drive
# # 將自己的雲端硬碟掛載上去
# drive.mount('/content/gdrive')

# os.chdir('./gdrive/MyDrive/Colab Notebooks/cheng_ta/final/dataset')      # 檔案目錄

In [2]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd
import os
import pickle
import random
import time
import importlib
import sys
sys.path.append('./GWFM/')
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from methods.DataIO import StructuralDataSampler, StructuralDataSampler2
import baselineModels as bm

# load data

In [3]:
with open('./dataset/datas.pickle', 'rb') as f:
    datas = pickle.load(f)

# node labels
Nclasses = []
for i in range(len(datas)):
    temp = torch.tensor(datas[i]['nodesLabel'], dtype=torch.long)
    Nclasses.append(temp)

classWeight = [0] * 38
for i in range(len(Nclasses)):
    for j in range(Nclasses[i].shape[0]):
        c = Nclasses[i][j]
        classWeight[c] += 1
classWeight = torch.tensor(classWeight, dtype=torch.float)
classWeight = classWeight / torch.sum(classWeight)
classWeight = 1 / classWeight
classWeight = classWeight / torch.sum(classWeight)
del Nclasses
classWeight.shape

torch.Size([38])

# model define

In [4]:
modelSave = './models/baseline/GCNnodeModel.pt'
lossSave = './models/baseline/nodeLoss_baseline.pickle'
# 訓練超參數 model
data_sampler = StructuralDataSampler2(datas)
num_samples = len(data_sampler)
inputD = 4
outputD = 38
num_classes = None       # 先驗分布
prior = None             # 先驗分布

# 訓練超參數 train
size_batch = 16
epochs = 100
lr = 0.1
weight_decay = 0
shuffle_data = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# importlib.reload(bm)
model = bm.GCN(inputD, 0.1).to(device)

In [7]:
def train_baseline(model,
                  data_sampler,
                  size_batch: int = 16,
                  epochs: int = 10,
                  lr: float = 1e-1,
                  weight_decay: float = 0,
                  shuffle_data: bool = True,
                  device=None):
    """
    training a FGWF model
    Args:
        model: a FGWF model
        database: a list of data, each element is a list representing [cost, distriubtion, feature, label]
        size_batch: the size of batch, deciding the frequency of backpropagation
        epochs: the number epochs
        lr: learning rate
        weight_decay: the weight of the l2-norm regularization of parameters
        shuffle_data: whether shuffle data in each epoch
        zeta: the weight of the regularizer enhancing the diversity of atoms
        mode: fit or transform
        visualize_prefix: display learning result after each epoch or not
    """
    global modelSave, lossSave, classWeight
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)  
    lossfun = nn.CrossEntropyLoss(weight=classWeight).to(device)
    model.train()

    num_samples = data_sampler.__len__()
    index_samples = list(range(num_samples))
    index_split = int(num_samples * 0.8)
    
    random.shuffle(index_samples)
    index_train = index_samples[:index_split]
    index_val = index_samples[index_split:]
    
    epoch_metric = {'loss':[], 'accu':[], 'f1':[]}
    
    t_start = time.time()

    for epoch in range(epochs):
        batch_loss = []

        if shuffle_data:
            random.shuffle(index_train)
        
        total_loss = 0.
        counts = 0
        for i in index_train:
            data = data_sampler[i]
            adj, features, labels = data[0].to(device), data[2].to(device), data[4].to(device)
            y = model(features, adj)
            loss = lossfun(y, labels)
            total_loss += loss
            
            counts += 1
            if counts % size_batch == 0 or counts == num_samples:
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                batch_loss.append(total_loss.item())
                total_loss = 0.
                counts = 0
            
        el = sum(batch_loss) / len(batch_loss)
        epoch_metric['loss'].append(el)
        
        preds = []
        trues = []
        for i in index_val:
            with torch.no_grad():
                data = data_sampler[i]
                adj, features, labels = data[0].to(device), data[2].to(device), data[4].to(device)
                y = model(features, adj)
            pred = torch.argmax(y, dim=1).cpu().detach()
            true = labels.cpu().detach()
            preds.append(pred)
            trues.append(true)
        preds = torch.cat(preds, dim=0)
        trues = torch.cat(trues, dim=0)
            
        accu = accuracy_score(preds, trues)
        f1 = f1_score(preds, trues, average='macro')
        epoch_metric['accu'].append(accu)
        epoch_metric['f1'].append(f1)
        print('epoch loss: {}, accu/f1:{}/{}, epoch:{}/{}, time:{}'.format(el, accu, f1, epoch, epochs, time.time()-t_start))
        
        with open(lossSave, 'wb') as f:
              pickle.dump(epoch_metric, f)
        torch.save(model.state_dict(), modelSave)

In [9]:
# train_baseline(model=model,
#               data_sampler=data_sampler,
#               size_batch = size_batch,
#               epochs = epochs,
#               lr = lr,
#               weight_decay = weight_decay,
#               shuffle_data = shuffle_data,
#               device=device)

In [10]:
with open('./models/baseline/nodeLoss_baseline.pickle', 'rb') as f:
    nodeLoss_baseline = pickle.load(f)

In [13]:
max(nodeLoss_baseline['accu']), max(nodeLoss_baseline['f1'])

(0.6007776049766719, 0.06697287885806016)